# Azure ML service による Rモデル開発

## Python SDK インポート

In [1]:
from azureml.core import Workspace, Experiment
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.21


## Azure Machine Learning service ワークスペース接続

In [2]:
ws = Workspace.get('azureml', subscription_id='9c0f91b8-eb2f-484c-979c-15848c098a6b', resource_group='dllab')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
# Experiment 設定
experiment = Experiment(workspace = ws, name = "R_training")

## Machine Learning Compute (計算環境) の構築

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_target = ComputeTarget(workspace=ws, name="cpucluster")

## 学習スクリプトの設定

In [5]:
script_params = {
    '--f': 'train.R',
    '--alpha': 14,
    '--beta': 10,
}

In [6]:
from azureml.train.estimator import Estimator
est = Estimator(source_directory='.', 
                compute_target=compute_target,
                conda_packages=['r-essentials', 'mro-base', 'r-reticulate'],
                entry_script='launchR.py', 
                script_params=script_params)

## 学習開始

In [7]:
run = experiment.submit(est)

In [8]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## ハイパーパラメータチューニング

In [9]:
experiment = Experiment(workspace = ws, name = "R_Hyperdrive")

In [10]:
script_params = {
    '--f': 'train.R'
}

In [11]:
from azureml.train.estimator import Estimator
est2 = Estimator(source_directory='.', 
                compute_target=compute_target,
                conda_packages=['r-essentials', 'mro-base', 'r-reticulate'],
                entry_script='launchR.py', 
                script_params=script_params)

In [15]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveRunConfig, uniform, PrimaryMetricGoal

param_sampling = RandomParameterSampling( {
    '--alpha': uniform(1, 100),
    '--beta': uniform(-100, 100)
    }
)

hyperdrive_run_config = HyperDriveRunConfig(estimator=est2,
                                            hyperparameter_sampling=param_sampling, 
                                            primary_metric_name='AUC',
                                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs=8,
                                            max_concurrent_runs=2)

In [16]:
htr = experiment.submit(config=hyperdrive_run_config)

In [17]:
RunDetails(htr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…